In [ ]:
import os
import pandas as pd
import music21
import numpy as np

"Below is the code to transform a MIDI file into a CSV. To use it, ensure that the MIDI file is located in the same folder as this notebook. Then, in the third code cell, change the MIDI file name to the name of the file you want to process.

This notebook uses the `midi_to_csv` function originally developed by Andrew Chen, available at [https://github.com/andrewchenk](https://github.com/andrewchenk)."

In [ ]:
def midi_to_csv(input_dir_name, output_dir_name=None):
    input_dir = os.fsencode(input_dir_name)
    
    if not output_dir_name:
        output_dir_name = input_dir_name + "-output-csvs"
    
    if not os.path.exists(output_dir_name):
        os.makedirs(output_dir_name)
    
    print("Outputting CSV files into " + output_dir_name)
    
    for file in os.listdir(input_dir):
        filename = os.fsdecode(file)
        
        # Filtra apenas arquivos MIDI
        if not filename.endswith(".mid"):
            continue
        
        print("Processing " + filename + " into " + filename[:-4] + ".csv")
        
        mf = music21.midi.MidiFile()
        mf.open(input_dir_name + "/" + filename)
        mf.read()
        mf.close()
        s = music21.midi.translate.midiFileToStream(mf, quantizePost=False).flat
        
        df = pd.DataFrame(columns=["note_name", "start_time", "duration", "velocity", "tempo"])
        for g in s.recurse().notes:
            if g.isChord:
                for pitch in g.pitches: 
                    x = music21.note.Note(pitch, duration=g.duration)
                    x.volume.velocity = g.volume.velocity
                    x.offset = g.offset
                    s.insert(x)
        
        note_tempo = s.metronomeMarkBoundaries()[0][2].number
        for note in s.recurse().notes: 
            if note.isNote:
                new_df = pd.DataFrame([[note.pitch, round(float(note.offset), 3), round(note.duration.quarterLength, 3), note.volume.velocity, note_tempo]],
                                      columns=["note_name", "start_time", "duration", "velocity", "tempo"])
                df = pd.concat([df, new_df], ignore_index=True)
        
        output_csv_file = output_dir_name + "/" + filename[:-4] + ".csv"
        df.to_csv(output_csv_file, index=False)
        
        # Modify the CSV to your standards
        modify_csv(output_csv_file, output_csv_file)

    print("Done creating and modifying CSVs!")


def modify_csv(input_file, output_file):
    """ Modifies a CSV file generated from a MIDI file by transforming and reformatting its content.
    Parameters:
    input_file : str
        Path to the input CSV file that needs to be modified.
    output_file : str
        Path where the modified CSV file will be saved. """
    df = pd.read_csv(input_file)
    # Replace '-' with 'b' in the note_name column
    df['note_name'] = df['note_name'].str.replace('-', 'b')
    def note_to_pitch(note_name):
        note_to_pitch_dict = {
        'C0': 12, 'C#0': 13, 'Db0': 13, 'D0': 14, 'D#0': 15, 'Eb0': 15, 'E0': 16, 'F0': 17, 'F#0': 18, 'Gb0': 18, 'G0': 19, 'G#0': 20, 'Ab0': 20, 'A0': 21, 'A#0': 22, 'Bb0': 22, 'B0': 23,
        'C1': 24, 'C#1': 25, 'Db1': 25, 'D1': 26, 'D#1': 27, 'Eb1': 27, 'E1': 28, 'F1': 29, 'F#1': 30, 'Gb1': 30, 'G1': 31, 'G#1': 32, 'Ab1': 32, 'A1': 33, 'A#1': 34, 'Bb1': 34, 'B1': 35,
        'C2': 36, 'C#2': 37, 'Db2': 37, 'D2': 38, 'D#2': 39, 'Eb2': 39, 'E2': 40, 'F2': 41, 'F#2': 42, 'Gb2': 42, 'G2': 43, 'G#2': 44, 'Ab2': 44, 'A2': 45, 'A#2': 46, 'Bb2': 46, 'B2': 47,
        'C3': 48, 'C#3': 49, 'Db3': 49, 'D3': 50, 'D#3': 51, 'Eb3': 51, 'E3': 52, 'F3': 53, 'F#3': 54, 'Gb3': 54, 'G3': 55, 'G#3': 56, 'Ab3': 56, 'A3': 57, 'A#3': 58, 'Bb3': 58, 'B3': 59,
        'C4': 60, 'C#4': 61, 'Db4': 61, 'D4': 62, 'D#4': 63, 'Eb4': 63, 'E4': 64, 'F4': 65, 'F#4': 66, 'Gb4': 66, 'G4': 67, 'G#4': 68, 'Ab4': 68, 'A4': 69, 'A#4': 70, 'Bb4': 70, 'B4': 71,
        'C5': 72, 'C#5': 73, 'Db5': 73, 'D5': 74, 'D#5': 75, 'Eb5': 75, 'E5': 76, 'F5': 77, 'F#5': 78, 'Gb5': 78, 'G5': 79, 'G#5': 80, 'Ab5': 80, 'A5': 81, 'A#5': 82, 'Bb5': 82, 'B5': 83,
        'C6': 84, 'C#6': 85, 'Db6': 85, 'D6': 86, 'D#6': 87, 'Eb6': 87, 'E6': 88, 'F6': 89, 'F#6': 90, 'Gb6': 90, 'G6': 91, 'G#6': 92, 'Ab6': 92, 'A6': 93, 'A#6': 94, 'Bb6': 94, 'B6': 95,
        'C7': 96, 'C#7': 97, 'Db7': 97, 'D7': 98, 'D#7': 99, 'Eb7': 99, 'E7': 100, 'F7': 101, 'F#7': 102, 'Gb7': 102, 'G7': 103, 'G#7': 104, 'Ab7': 104, 'A7': 105, 'A#7': 106, 'Bb7': 106, 'B7': 107,
        'C8': 108, 'C#8': 109, 'Db8': 109, 'D8': 110, 'D#8': 111, 'Eb8': 111, 'E8': 112, 'F8': 113, 'F#8': 114, 'Gb8': 114, 'G8': 115, 'G#8': 116, 'Ab8': 116, 'A8': 117, 'A#8': 118, 'Bb8': 118, 'B8': 119,
        'C9': 120, 'C#9': 121, 'Db9': 121, 'D9': 122, 'D#9': 123, 'Eb9': 123, 'E9': 124, 'F9': 125, 'F#9': 126, 'Gb9': 126, 'G9': 127, 'G#9': 128, 'Ab9': 128, 'A9': 129, 'A#9': 130, 'Bb9': 130, 'B9': 131,
        'C10': 132, 'C#10': 133, 'Db10': 133, 'D10': 134, 'D#10': 135, 'Eb10': 135, 'E10': 136, 'F10': 137, 'F#10': 138, 'Gb10': 138, 'G10': 139, 'G#10': 140, 'Ab10': 140, 'A10': 141, 'A#10': 142, 'Bb10': 142, 'B10': 143
        }
        return note_to_pitch_dict.get(note_name, None)

    # Apply the transformation to each row
    df['Pitch'] = df['note_name'].apply(note_to_pitch)
    df['Start'] = df['start_time'].round(2)
    df['Duration'] = df['duration'].round(2)
    df['Velocity'] = df['velocity'] / 127  # Transform velocity to standards of libfmp

    # Select and reorder columns
    df = df[['Start', 'Duration', 'Pitch', 'Velocity']]

    # Add the 'instrument' column with NaN values
    df['instrument'] = np.nan

    # Save the DataFrame to a new CSV file
    df.to_csv(output_file, index=False, sep=';', header=True)

    print(f"CSV file '{output_file}' has been generated.")


In [ ]:
# Name of the MIDI file to be processed (change the name to the desired file)
midi_filename = "filename.mid"

# Output directory for the generated CSV files
output_dir = "csv_output"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

midi_to_csv(input_dir_name=".", output_dir_name=output_dir)

print(f"Processing complete! The file '{midi_filename}' has been converted and saved in '{output_dir}'.")
